# Python `tspart` Module Usage Example

## Step 0: Initial setup tools.

### Install the `tspart` module.

In [ ]:
!pip install --upgrade --no-cache-dir tspart

### Download the default test image.

In [ ]:
!curl https://raw.githubusercontent.com/nimaid/python-tspart/main/tests/baboon.png -o baboon.png
!mkdir tests
!mv baboon.png tests/baboon.png

## Step 1: Setup environment.

In [ ]:
import os
import tspart
from IPython.display import display

def display_width(img, width=600):
    ratio = img.size[0] / img.size[1]
    
    new_size = (
        width,
        int(round(width / ratio))
    )
    
    display(img.resize(new_size))

In [ ]:
working_directory = "tests"
image_file = os.path.join(working_directory, "baboon.png")
number_of_points = 500

image_name = os.path.splitext(os.path.basename(image_file))[0]
tsp_image_file = os.path.join(working_directory, f"{image_name}_{number_of_points}.png")
pre_tsp_file = os.path.join(working_directory, f"{image_name}_{number_of_points}_raw.tspart")
tsp_file = os.path.join(working_directory, f"{image_name}_{number_of_points}.tspart")
jobs_file = os.path.join(working_directory, f"{image_name}_{number_of_points}.jobs")

In [ ]:
email = input("Please enter your email address (for sending requests to the Neos online solver):").strip()

## Step 2: Import the image and get it's size.

In [ ]:
img = tspart.load_image_as_array(image_file)
size = tspart.image_array_size(img)

## Step 3: Prepare the monochrome image(s).

#### Monochrome Option: Convert to black and white.

In [ ]:
img_channels = [tspart.rgb_array_to_grayscale(img)]

#### CMYK Option: Split channels.

In [ ]:
img_channels = tspart.split_cmyk(img)

#### RGB Option: Split channels.

In [ ]:
img_channels = tspart.split_rgb(img, invert=True)

## Step 4: Stipple the images.
This can a few hours and a lot of memory for larger numbers of points.

In [ ]:
points = tspart.voronoi.stipple_image_multi(img_channels, points=number_of_points)

#### Optional: Filter out points that lie on pure white pixels.

In [ ]:
points = tspart.filter_white_points_multi(img_channels, points)

### Step 4.5 (optional): Save the stippled points to disk (in case of failure).

In [ ]:
tspart.save_tspart(
    filename=pre_tsp_file,
    points=points,
    factors=[[1.0 for __ in _] for _ in points],
    size=size
)

## Step 5: Solve the TSP problems with a free online solver.
This will take a long time.

#### Part A: Create the client and submit the jobs for processing

In [ ]:
neos = tspart.neos.get_client()

In [ ]:
jobs = tspart.neos.submit_solves(neos, email, points)

##### Optional: Save the jobs to disk (in case of failure).

In [ ]:
tspart.save_jobs(jobs_file, jobs)

#### Part B: Wait for results.

In [ ]:
routes = tspart.neos.get_solves_blocking(neos, jobs, points)

#### Part C: Compute size factors.

In [ ]:
factors = tspart.factors_from_image_multi(img_channels, routes)

### Step 5.5 (optional): Save the routes locally (in case of failure).
Additionally, delete the pre-TSP data.

In [ ]:
tspart.save_tspart(
    filename=tsp_file,
    points=routes,
    factors=factors,
    size=size
)

if os.path.exists(pre_tsp_file):
    os.remove(pre_tsp_file)

if os.path.exists(jobs_file):
    os.remove(jobs_file)

## Step 6: Draw a bitmap image.
You can tweak the line width until you get a good looking result.

#### Monochrome Option

In [ ]:
drawn_image = tspart.draw_route(
    points=routes[0],
    factors=factors[0],
    size=size,
    line_width=12,
    scale=2
)

display_width(drawn_image)

#### CMYK Option

In [ ]:
drawn_image = tspart.draw_cmyk_routes(
    cmyk_points=routes,
    cmyk_factors=factors,
    size=size,
    line_width=8,
    scale=2
)

display_width(drawn_image)

#### RGB Option

In [ ]:
drawn_image = tspart.draw_rgb_routes(
    rgb_points=routes,
    rgb_factors=factors,
    size=size,
    line_width=8,
    scale=2
)

display_width(drawn_image)

## Step 7: Save the image.

In [ ]:
drawn_image.save(tsp_image_file)

# Tools:

## Alternate Step 4: Load raw (pre-TSP) points from disk.

In [ ]:
art_object = tspart.load_tspart(pre_tsp_file)

points = art_object["points"]
factors = art_object["factors"]
size = art_object["size"]

## Alternate Step 5 (A): Recover Neos solves with saved jobs file.

In [ ]:
neos = tspart.neos.get_client()
jobs = tspart.load_jobs(jobs_file)

## Alternate Step 5: Load precomputed routes from disk.

In [ ]:
art_object = tspart.load_tspart(tsp_file)

routes = art_object["points"]
factors = art_object["factors"]
size = art_object["size"]

## Alternate Step 5: Run a heuristic (non-exact) solve locally
A 1-hour timeout is used by default for each channel.

In [ ]:
routes = tspart.tsp.heuristic_solves(points, verbose=True)

## Print the job numbers and passwords for the current jobs.

In [ ]:
print(jobs)

## Cancel Neos solves and delete jobs file.

In [ ]:
tspart.neos.cancel_solves(neos, jobs)

if os.path.exists(jobs_file):
    os.remove(jobs_file)